# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when KRAS has missense mutations compared to KRAS wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
def get_all_interacting_proteins(gene, num_results_in = 25):
    bioplex_list = list(u.get_interacting_proteins_bioplex(gene))
    wiki_list = list(u.get_interacting_proteins_wikipathways(gene))

    biogrid_list = list(u.get_interacting_proteins_biogrid(gene, num_results=num_results_in))
    string_list = list(u.get_interacting_proteins_string(gene, num_results=num_results_in))

    all_int = bioplex_list + wiki_list + biogrid_list + string_list
    interacting_list = list(set(all_int))
    interacting_list.remove(gene) # remove cis
    
    return interacting_list

In [33]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, all_prot, gene_in = 'KRAS', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    mut_type['Mutation'].where(mut_type['Mutation'] != 'Missense_Mutation', 'Missense', inplace = True) # replace when false
    mut_type['Mutation'].where(mut_type['Mutation'] != 'nonsynonymous SNV', 'Missense', inplace = True)

    # Keep only tumor samples from proteomics
    prot_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
    # Reduce a multiindex 
    if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
        prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
        prot_and_mutations = rename_duplicate_cols(prot_and_mutations)# make unique col names

    # Keep certain missense mutations
    prot_and_mutations['KRAS_Location'] = [','.join(map(str, l)) for l in prot_and_mutations['KRAS_Location']]
    hotspots = ['G12', 'G13', 'Q61', 'No_mutation']
    hotspots_wt = pd.DataFrame()
    for site in hotspots:
        df = prot_and_mutations[prot_and_mutations.KRAS_Location.str.contains(site, regex= True, na=False)]
        hotspots_wt = hotspots_wt.append(df)
    #print(hotspots_wt.KRAS_Location.value_counts())
    
    # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
    mut_type = mut_type[['Mutation']]
    prot_df = hotspots_wt.iloc[:,:-4] # Keep only proteomics
    merged = prot_df.join(mut_type)
    merged = merged.join(hotspots_wt['KRAS_Location'])

    # Keep only Wildtype and deletion
    compare = ['Wildtype_Tumor','Missense']
    get = merged['Mutation'].isin(compare)
    mut_wt = merged[get]
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [5]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    d = del_wt_df[del_wt_df.Mutation == "Missense"]
    wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

In [6]:
# Adds '_i' to duplicate col names where i is a number (increases with each duplicate)

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [7]:
en = cptac.Endometrial()
l = cptac.Luad()
col = cptac.Colon()

# Calculate p-values and differential expressions

In [26]:
cancer_objects = {'Endo':en, 'Colon':col, 'Luad':l}

In [52]:
# Get total num prot cols
total_prot = {}
for c in cancer_objects:
    df = cancer_objects[c].get_proteomics()
    total_prot[c] = len(df.columns)

In [53]:
import warnings
warnings.filterwarnings('ignore')

# Get interacting
prot_list = get_all_interacting_proteins(gene, 100)
#prot_list = prot_list.remove(gene) # cis effect
print('Interacting genes:', len(prot_list))

gene = 'KRAS'
merged_dfs = {}
for cancer in cancer_objects:
    print(cancer)

    # Format df for t-test
    mut_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    mut_wt = mut_wt.dropna(axis = 'columns', how = 'all') #drop cols with only nan
    
    # T-test
    cols_in = list(mut_wt.columns[:-2])
    print('cols tested: ', len(cols_in), '/', total_prot[cancer])

    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(mut_wt, 'Mutation', cols_in, return_all = True, correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get difference in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, mut_wt, cols_in)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Proteomics', how='outer')
    pval_medians_df = pval_medians_df.replace(to_replace = r'_proteomics', value = '', regex = True)
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    #pval_medians_df.to_csv('csv/Single_Cancer/'+cancer+'_pval_medians_interacting.csv', index=False)
    print(pval_medians_df.head(), '\n')

Interacting genes: 14322
Endo
cols tested:  8955 / 10999
  Proteomics  Endo_P_Value  Endo_Median
0    ZFYVE19      0.007052      0.34000
1      MUC5B      0.008843      2.23350
2      FSTL1      0.009692      1.01400
3   CDC42SE2      0.013058      0.45250
4     MUC5AC      0.033080      1.67655 

Colon
cols tested:  6783 / 8067
  Proteomics  Colon_P_Value  Colon_Median
0       IL33       0.717738        0.6220
1       GHDC       0.717738       -0.2154
2       CCNH       0.717738       -0.8540
3     EPSTI1       0.776265       -0.5040
4       ESPN       0.785796       -0.6780 

Luad
cols tested:  8601 / 10699
  Proteomics  Luad_P_Value  Luad_Median
0      DOCK5      0.001568      1.09795
1      TUFT1      0.001698      1.60680
2         F3      0.002924      2.78775
3     CYP4X1      0.003684     -2.68340
4       MPP6      0.006822     -1.22155 



# Get number of significant genes in each cancer

In [54]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

Endo sig comparisons: 8
Colon sig comparisons: 0
Luad sig comparisons: 47

Number of significant comparisons in at least 1 cancer: 55


In [45]:
e_df = merged_dfs['Endo']
e_df.loc[e_df['Endo_P_Value'] < 0.05]

,Proteomics,Endo_P_Value,Endo_Median
0,ZFYVE19,0.007052,0.34000
1,MUC5B,0.008843,2.23350
2,FSTL1,0.009692,1.01400
3,CDC42SE2,0.013058,0.45250
4,MUC5AC,0.033080,1.67655
5,SPRY4,0.046892,0.61115
6,SYTL1,0.049006,0.68075
7,RGL2,0.049006,0.16875


In [50]:
l_df = merged_dfs['Luad']
l_sig = l_df.loc[l_df['Luad_P_Value'] < 0.05]
l_sig

,Proteomics,Luad_P_Value,Luad_Median
0,DOCK5,0.001568,1.09795
1,TUFT1,0.001698,1.60680
2,F3,0.002924,2.78775
3,CYP4X1,0.003684,-2.68340
4,MPP6,0.006822,-1.22155
5,FAM83A,0.009744,3.05200
6,IVNS1ABP,0.011210,0.90520
7,ERI3,0.011210,-0.62310
8,POM121C,0.011210,-0.72425
9,LCOR,0.011210,1.19465


# Get a list of significant genes in at least one cancer

In [37]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
#s.to_csv('csv/list_sig_one_cancer_interacting.csv', index=False)

Number of significant genes in at least one cancer: 55


# Get a list of significant genes in multiple cancers

In [38]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

Number of significant genes in mult cancers: 0


In [20]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
#m.to_csv('csv/list_sig_multiple_cancers_interacting.csv', index=False)